In [43]:
import pandas as pd
import numpy as np

import os

import matplotlib.pyplot  as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import env

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [44]:
def get_zillow():
    '''
    This function reads in zillow data from Codeup database using sql querry into a dataframe and returns a dataframe. 
    '''
        
    query = """
            SELECT * FROM 
            properties_2017
            left join predictions_2017 USING(parcelid)
            left join airconditioningtype USING(airconditioningtypeid)
            left join architecturalstyletype USING(architecturalstyletypeid)
            left join buildingclasstype USING(buildingclasstypeid)
            left join heatingorsystemtype USING(heatingorsystemtypeid)
            left join propertylandusetype USING(propertylandusetypeid)
            left join storytype USING(storytypeid)
            left join typeconstructiontype USING(typeconstructiontypeid)
            left join unique_properties USING(parcelid)
            where transactiondate like '2017%%';
            """
    
    filename = "zillow_2017.csv"
    
    # check if a file exits in a local drive
    if os.path.isfile(filename):
        
        #if yes, read data from a file
        df = pd.read_csv(filename)
        
    else:
        # If not, read data from database into a dataframe
        df = pd.read_sql(query, env.get_connection('zillow'))

        # Write that dataframe in to disk for later, Called "caching" the data for later.
        df.to_csv(filename, index=False)
     
    #returns dataframe
    return df

In [45]:
df = get_zillow()

df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,3.5,4.0,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,NaN,122,NaN,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,1.0,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,NaN,NaN,6.059063e+13,0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,1.0,2.0,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,NaN,1110,NaN,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,NaN,NaN,6.111001e+13,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,2.0,3.0,NaN,2.0,NaN,NaN,1243.0,1243.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,1.0,122,NaN,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,NaN,NaN,6.059022e+13,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,3.0,4.0,8.0,3.0,NaN,NaN,2376.0,2376.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,1.0,0101,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,NaN,NaN,6.037300e+13,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,3.0,3.0,8.0,3.0,NaN,NaN,1312.0,1312.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34185120.0,-118414640.0,278581.0,1.0,NaN,NaN,NaN,1.0,010C,LAR3,6.037124e+07,12447.0,3101.0,268548.0,96451.0,0.0,NaN,1.0,NaN,NaN,1964.0,NaN,NaN,73681.0,119407.0,2016.0,45726.0,1533.89,NaN,NaN,6.037124e+13,4,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [46]:
df.shape

(77613, 69)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77613 entries, 0 to 77612
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77613 non-null  int64  
 1   typeconstructiontypeid        223 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77579 non-null  float64
 4   heatingorsystemtypeid         49571 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      207 non-null    float64
 7   airconditioningtypeid         25007 non-null  float64
 8   id                            77613 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77579 non-null  float64
 11  bedroomcnt                    77579 non-null  float64
 12  buildingqualitytypeid         49809 non-null  float64
 13  c

In [48]:
df= df.sort_values('id')

In [49]:
bool_id = df['id'].duplicated()
bool_id.value_counts()

False    77414
True       199
Name: id, dtype: int64

In [50]:
# df = df[~bool_id]
# df

In [51]:
df= df.sort_values('transactiondate')

In [52]:
bool_transactiondate1 = df['transactiondate'].duplicated(keep='first')
bool_transactiondate1 .value_counts()

# True     77349
# False      264

True     77349
False      264
Name: transactiondate, dtype: int64

In [53]:
bool_transactiondate2 = df['transactiondate'].duplicated(keep='last')
bool_transactiondate2 .value_counts()
# True     77349
# False      264

True     77349
False      264
Name: transactiondate, dtype: int64

In [54]:
# bool_transactiondate3 = df['logerror'].duplicated(keep='last')
# bool_transactiondate3 .value_counts()
# False    77064
# True       549

In [55]:
df = df[bool_transactiondate1]
df.shape

(77349, 69)

In [56]:
# df = df[~bool_transactiondate3]
# df.shape

In [57]:
boolean_1= ((pd.isnull(df['latitude'])) & (pd.isnull(df['longitude'])))

In [58]:
df= df[~boolean_1]
df.shape

(77315, 69)

In [59]:
df.latitude.isnull().sum()


0

In [60]:
df.longitude.isnull().sum()


0

In [61]:
df.longitude.isnull().sum()

0

In [62]:
df.shape

(77315, 69)

In [63]:
df.parcelid.isnull().sum()

0

In [64]:
df.airconditioningtypeid.isnull().sum()

52410

In [65]:
df.architecturalstyletypeid.isnull().sum()

77109

In [66]:
def row_missing(df):
    num_rows_missing = []
    percent_rows_missing = []
    index = df.columns.tolist()
    for col in df.columns:
        num_rows_missing.append(df[col].isnull().sum())
        percent_rows_missing.append(((df[col].isnull().sum())/len(df))* 100)
        
    metric_df = pd.DataFrame({'num_row_missing': num_rows_missing,'pct_rowss_missing':percent_rows_missing},index=index)
    return metric_df
        

In [67]:
nan_df = row_missing(df)
nan_df.head(10)

,num_row_missing,pct_rowss_missing
parcelid,0,0.000000
typeconstructiontypeid,77093,99.712863
storytypeid,77265,99.935329
propertylandusetypeid,0,0.000000
heatingorsystemtypeid,27916,36.106836
buildingclasstypeid,77300,99.980599
architecturalstyletypeid,77109,99.733558
airconditioningtypeid,52410,67.787622
id,0,0.000000
basementsqft,77265,99.935329


In [68]:
df = df[df['propertylandusetypeid']== 261]
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,3.0,4.0,8.0,3.0,NaN,NaN,2376.0,2376.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,1.0,0101,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,NaN,NaN,6.037300e+13,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,1.0,2.0,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,NaN,1110,NaN,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,NaN,NaN,6.111001e+13,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
7,12069064,NaN,NaN,261.0,NaN,NaN,NaN,NaN,870991,NaN,1.0,2.0,5.0,1.0,NaN,NaN,738.0,738.0,NaN,NaN,NaN,NaN,6037.0,NaN,1.0,NaN,NaN,NaN,34149214.0,-118239357.0,4214.0,NaN,NaN,NaN,NaN,NaN,0100,GLR4YY,6.037302e+07,45457.0,3101.0,276258.0,96325.0,0.0,NaN,1.0,NaN,NaN,1922.0,NaN,NaN,18890.0,218552.0,2016.0,199662.0,2366.08,NaN,NaN,6.037302e+13,7,0.101723,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
6,12095076,NaN,NaN,261.0,2.0,NaN,NaN,1.0,781532,NaN,3.0,4.0,9.0,3.0,NaN,NaN,2962.0,2962.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34145202.0,-118179824.0,63000.0,1.0,NaN,NaN,NaN,1.0,0101,PSR2,6.037461e+07,47019.0,3101.0,274684.0,96293.0,0.0,NaN,1.0,NaN,NaN,1950.0,NaN,NaN,276684.0,773303.0,2016.0,496619.0,9516.26,NaN,NaN,6.037461e+13,6,-0.001011,2017-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,2.0,3.0,NaN,2.0,NaN,NaN,1243.0,1243.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,1.0,122,NaN,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,NaN,NaN,6.059022e+13,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN


In [69]:
df.propertylandusetypeid.value_counts()

261.0    52249
Name: propertylandusetypeid, dtype: int64

In [70]:
def remove_columns(df, cols_to_remove):  
    df = df.drop(columns=cols_to_remove)
    return df


In [71]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df


In [72]:
query = '''
        SELECT  * FROM 
        customers;
        '''

customers = pd.read_sql(query, env.get_connection('mall_customers'))

In [73]:
customers.head()

,customer_id,gender,age,annual_income,spending_score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [74]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     200 non-null    int64 
 1   gender          200 non-null    object
 2   age             200 non-null    int64 
 3   annual_income   200 non-null    int64 
 4   spending_score  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB


In [75]:
for col in customers.select_dtypes(exclude='object'):
    q1, q3 = customers[col].quantile([.25, .75])
    iqr = q3 - q1 
    upper_bound = q3 + 1.5 * iqr
    lower_bound = q1- 1.5 * iqr
    print(col)
    print(f' upper bound: {upper_bound}, lower bound: {lower_bound}\n') 

    
    
    
    

customer_id
 upper bound: 299.5, lower bound: -98.5

age
 upper bound: 79.375, lower bound: -1.625

annual_income
 upper bound: 132.75, lower bound: -13.25

spending_score
 upper bound: 130.375, lower bound: -22.625



In [76]:
def train_val_test(df, seed=42):
    '''split data into train, validate and test data'''
    
    # split data into 80% train_validate, 20% test
    train_validate, test = train_test_split(df, test_size=0.2, 
                                            random_state=seed)
    
    # split train_validate data into 70% train, 30% validate
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=seed)
    
    # returns train, validate, test data
    return train, validate, test


In [77]:
train, validate, test = train_val_test(customers, seed=42)
train.shape, validate.shape, test.shape

((112, 5), (48, 5), (40, 5))

In [78]:
def make_dummies(df, target):
    dummies = pd.get_dummies(df[target],drop_first=True)
    df = pd.concat([df,dummies],axis=1)
    return df
    

In [81]:
train_encoded = make_dummies(train, 'gender')
train_encoded.head()

,customer_id,gender,age,annual_income,spending_score,Male
130,131,Male,47,71,9,1
62,63,Female,67,47,52,0
13,14,Female,24,20,77,0
173,174,Male,36,87,92,1
144,145,Male,25,77,12,1
